<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/tunning/CB_divide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import warnings
# 경고 제거
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product.parquet')

In [22]:
product.head()

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,27781
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,20698
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,27546
3,59262,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,27546
4,59223,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,27546


In [44]:
# 중복값제거 
product_c =product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)

In [46]:
product_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31121 entries, 0 to 31120
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   product_id          31121 non-null  uint16  
 1   gender              31121 non-null  category
 2   masterCategory      31121 non-null  category
 3   subCategory         31121 non-null  category
 4   articleType         31121 non-null  category
 5   baseColour          31115 non-null  category
 6   season              31100 non-null  category
 7   year                31121 non-null  int16   
 8   usage               30824 non-null  category
 9   productDisplayName  31121 non-null  category
 10  p_idx               31121 non-null  int32   
dtypes: category(8), int16(1), int32(1), uint16(1)
memory usage: 1.8 MB


In [26]:
product_c['gender'].value_counts()

Men       15426
Women     12974
Unisex     1748
Boys        538
Girls       435
Name: gender, dtype: int64

In [27]:
product_c['masterCategory'].value_counts()

Apparel           15455
Accessories        6710
Footwear           6606
Personal Care      2263
Free Items           61
Sporting Goods       25
Home                  1
Name: masterCategory, dtype: int64

In [6]:
#결측값 제거
df = product_c.copy()

In [7]:
df.dropna(subset=['productDisplayName','year'],inplace=True)

In [8]:
col = ['baseColour','season','usage']
df[col]=df[col].astype('object')
df[col]=df[col].fillna("unknown")
df[col]=df[col].astype('category')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31121 entries, 0 to 31120
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   product_id          31121 non-null  uint16  
 1   gender              31121 non-null  category
 2   masterCategory      31121 non-null  category
 3   subCategory         31121 non-null  category
 4   articleType         31121 non-null  category
 5   baseColour          31121 non-null  category
 6   season              31121 non-null  category
 7   year                31121 non-null  int16   
 8   usage               31121 non-null  category
 9   productDisplayName  31121 non-null  category
 10  p_idx               31121 non-null  int32   
dtypes: category(8), int16(1), int32(1), uint16(1)
memory usage: 2.0 MB


In [9]:
df['features'] = df[['gender','articleType','baseColour','season','usage']].apply(' '.join, axis=1)

## 성별로 나누기
- 남자(Men, Boys, Unisex)
- 여자(Women, Girls, Unisex)
- unisex는 그냥 다 넣어준다.

In [10]:
men = ['Men','Boys','Unisex']
women = ['Women','Girls','Unisex']
Men = df[df['gender'].isin(men)]
Women = df[df['gender'].isin(women)]

In [11]:
Men.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17712 entries, 0 to 31119
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   product_id          17712 non-null  uint16  
 1   gender              17712 non-null  category
 2   masterCategory      17712 non-null  category
 3   subCategory         17712 non-null  category
 4   articleType         17712 non-null  category
 5   baseColour          17712 non-null  category
 6   season              17712 non-null  category
 7   year                17712 non-null  int16   
 8   usage               17712 non-null  category
 9   productDisplayName  17712 non-null  category
 10  p_idx               17712 non-null  int32   
 11  features            17712 non-null  object  
dtypes: category(8), int16(1), int32(1), object(1), uint16(1)
memory usage: 1.8+ MB


In [12]:
Men.gender.value_counts()

Men       15426
Unisex     1748
Boys        538
Girls         0
Women         0
Name: gender, dtype: int64

In [13]:
Women.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15157 entries, 2 to 31120
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   product_id          15157 non-null  uint16  
 1   gender              15157 non-null  category
 2   masterCategory      15157 non-null  category
 3   subCategory         15157 non-null  category
 4   articleType         15157 non-null  category
 5   baseColour          15157 non-null  category
 6   season              15157 non-null  category
 7   year                15157 non-null  int16   
 8   usage               15157 non-null  category
 9   productDisplayName  15157 non-null  category
 10  p_idx               15157 non-null  int32   
 11  features            15157 non-null  object  
dtypes: category(8), int16(1), int32(1), object(1), uint16(1)
memory usage: 1.7+ MB


In [14]:
Women.gender.value_counts()

Women     12974
Unisex     1748
Girls       435
Boys          0
Men           0
Name: gender, dtype: int64

# TF-idf를 이용한 컨텐츠 기반 추천시스템

## Men

In [15]:
tfidf = TfidfVectorizer()
tfidf_men = tfidf.fit_transform(Men['features'])
print(tfidf_men.shape)

(17712, 162)


In [16]:
cosine_men= pd.DataFrame(cosine_similarity(tfidf_men,tfidf_men),index = Men.p_idx, columns=Men.p_idx)
cosine_men.head()

p_idx,27781,20698,16433,22200,12874,8473,22387,26021,22136,9695,...,18248,8901,9921,20897,14146,18855,5581,11124,15896,22174
p_idx,,,,,,,,,,,,,,,,,,,,,
27781,1.000000,0.241344,0.136076,0.095408,0.095408,0.230989,0.534957,0.080002,0.147863,0.162814,...,0.026986,0.050088,0.028656,0.070065,0.160849,0.042510,0.520244,0.139100,0.059469,0.520244
20698,0.241344,1.000000,0.049622,0.179277,0.179277,0.074004,0.269838,0.075264,0.228504,0.059373,...,0.025388,0.094118,0.086966,0.131656,0.033436,0.129011,0.338982,0.214962,0.111747,0.338982
16433,0.136076,0.049622,1.000000,0.069825,0.069825,0.017846,0.046066,0.172923,0.244426,0.209382,...,0.019750,0.036657,0.020972,0.051277,0.117719,0.166744,0.191127,0.101802,0.043523,0.191127
22200,0.095408,0.179277,0.069825,1.000000,1.000000,0.104133,0.166428,0.105907,0.321534,0.083546,...,0.035724,0.132436,0.122373,0.185257,0.245915,0.419402,0.387808,0.302479,0.157242,0.387808
12874,0.095408,0.179277,0.069825,1.000000,1.000000,0.104133,0.166428,0.105907,0.321534,0.083546,...,0.035724,0.132436,0.122373,0.185257,0.245915,0.419402,0.387808,0.302479,0.157242,0.387808


## Women

In [17]:
tfidf = TfidfVectorizer()
tfidf_women = tfidf.fit_transform(Women['features'])
print(tfidf_women.shape)

(15157, 219)


In [18]:
cosine_women= pd.DataFrame(cosine_similarity(tfidf_women,tfidf_women),index = Women.p_idx, columns=Women.p_idx)
cosine_women.head()

p_idx,27546,13203,10610,17281,11501,4565,3466,27362,20828,6319,...,3392,17029,20308,3701,25084,17351,27551,14205,27138,10647
p_idx,,,,,,,,,,,,,,,,,,,,,
27546,1.000000,0.060089,0.063154,0.075981,0.031838,0.057922,0.071276,0.018574,0.129040,0.058240,...,0.057334,0.029560,0.030472,0.197270,0.174020,0.240261,0.608857,0.036063,0.055640,0.594559
13203,0.060089,1.000000,0.137102,0.164948,0.101139,0.125744,0.154734,0.059003,0.040545,0.059253,...,0.124467,0.030074,0.031002,0.061983,0.054678,0.075491,0.070537,0.036690,0.056608,0.068881
10610,0.063154,0.137102,1.000000,0.173361,0.106298,0.132158,0.162627,0.166419,0.042613,0.062275,...,0.130816,0.031608,0.124085,0.890819,0.057467,0.302157,0.074135,0.038562,0.059496,0.072394
17281,0.075981,0.164948,0.173361,1.000000,0.127887,0.158999,0.624880,0.074608,0.051268,0.074923,...,0.157385,0.316891,0.039200,0.078376,0.069138,0.542293,0.089192,0.046393,0.071579,0.087098
11501,0.031838,0.101139,0.106298,0.127887,1.000000,0.097491,0.119968,0.058521,0.021482,0.031395,...,0.096501,0.000000,0.016426,0.032841,0.028971,0.039998,0.282603,0.019440,0.029993,0.036496


# 추천하기

In [19]:
purchase=pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/cart_purchase.parquet')
purchase.head()

,created_at,customer_id,product_id,p_idx,event,c_idx
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968


In [20]:
purchase['values']=1

In [21]:
purchase_group=purchase.groupby(['c_idx'], as_index = False).count()

lower_list = list(purchase_group.query("values < 20").c_idx) #28295

upper = purchase[purchase['c_idx'].isin(purchase_group.query("values >= 20").c_idx)]
lower = purchase[purchase['c_idx'].isin(purchase_group.query("values < 20").c_idx)]

In [67]:
3039 in lower_list

True

In [74]:
recent=lower[lower['c_idx']==3039].sort_values(by='created_at',ascending=False)[:1]
recent.p_idx.values[0]

men_sim=cosine_men[recent.p_idx.values[0]].sort_values(ascending=False).index

product_c[product_c['p_idx'].isin(men_sim)][:10]

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,27781
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,20698
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,16433
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,22200
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,12874
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,8473
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,22387
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,26021
10,9204,Men,Footwear,Shoes,Casual Shoes,Black,Summer,2011,Casual,Puma Men Future Cat Remix SF Black Casual Shoes,22136
12,18653,Men,Footwear,Flip Flops,Flip Flops,Black,Fall,2011,Casual,Fila Men Cush Flex Black Slippers,9695


In [77]:
def sim_cosine(c_idx, k=25):
  try:
    if c_idx in lower_list: # 20개 이하 산 사람
      buy_recent = lower[lower['c_idx']==c_idx].sort_values(by='created_at',ascending=False)[:1]
      p_idx = buy_recent.p_idx.values[0]

      if p_idx in Men['p_idx']: # p_idx가 Men에 들어가있으면
        men_sim = cosine_men[p_idx].sort_values(ascending=False).index
        recommend = product_c[product_c['p_idx'].isin(men_sim)][:10]

      else: # 다른 경우
        women_sim = cosine_women[p_idx].sort_values(ascending=False).index[:10]
        recommend = product_c[product_c['p_idx'].isin(women_sim)]

    return recommend
  except:
    pass

In [78]:
sim_cosine(3039) #왜 안되지^^ -> 일단 try, except으로 막아둠 하나하나씩할땐 되는데;;;

In [68]:
sim_cosine(6934) 

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
2346,44771,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Globalite Men Navy Blue Skid Shoes,11720
4091,34858,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Vans Men Classic Slip-On Grey Shoes,29341
5544,7047,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2011,Casual,Numero Uno Men's Grey Blue Shoe,20091
6479,11834,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2013,Casual,Playboy Men Casual Grey Casual Shoes,20887
13834,44763,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Globalite Men Grey Startech Shoes,11718
14176,54739,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Vans Men Grey Del Norte Canvas Shoes,29361
21154,26645,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Puma Men Driving Power Light Grey Shoes,22016
24261,36478,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Spinn Men Agile Grey Shoes,26350
27627,42128,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2012,Casual,Nike Men Grey Avid Shoes,18550
27818,17055,Men,Footwear,Shoes,Casual Shoes,Grey,Summer,2013,Casual,Gas Men Count Casual Shoe,11125


In [64]:
sim_cosine(1743)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
507,43163,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Fastrack Men White Dial Watch,9145
908,40516,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Titan Men White Dial Watch,27512
6573,32283,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Q&Q Men White Dial Watch A150J204Y,23691
14531,29761,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Fossil Men White Dial Chronograph Watch CH2730,10597
14601,21706,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Fossil Men Quartz White Dial Watch FS4677,10582
14603,11193,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Carrera Men Steel dial frame dial White Watches,5047
16085,40558,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Titan Men White Dial Watch NB390YM14,27519
17986,41035,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Nautica Men White Dial Watch,17880
21569,25653,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Fastrack Men White Casual Watch,9143
28738,30043,Men,Accessories,Watches,Watches,White,Winter,2016,Casual,Skagen Men Silver Dial Watch,26026


In [66]:
sim_cosine(303)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
5492,59727,Women,Apparel,Innerwear,Bra,Red,Summer,2012,Casual,Avirate Red & Black Bra,3264
8533,21661,Women,Apparel,Innerwear,Bra,Red,Summer,2014,Casual,Enamor Women Smooth Red Bra,7814
12337,51194,Women,Apparel,Innerwear,Bra,Red,Summer,2014,Casual,Enamor Red Bra,7726
12962,21616,Women,Apparel,Innerwear,Bra,Red,Summer,2014,Casual,Enamor Red Smooth Bra,7727
14609,51829,Women,Apparel,Innerwear,Bra,Red,Summer,2016,Casual,Bwitch Red Bra,4595
15841,56298,Women,Apparel,Innerwear,Bra,Red,Summer,2017,Casual,Biara Women Red Blush Bra,4290
18940,54318,Women,Apparel,Innerwear,Bra,Red,Summer,2017,Casual,Garfield Women Pink & Red Bra,11097
19611,56422,Women,Apparel,Innerwear,Bra,Red,Summer,2017,Casual,Biara Red Blush Bra ST 5001,4270
26271,59706,Women,Apparel,Innerwear,Bra,Red,Summer,2014,Casual,Avirate Red & White Bra,3266
29344,50967,Women,Apparel,Innerwear,Bra,Red,Summer,2018,Casual,Enamor Glitzies Red Lift Bra DB05,7701


In [20]:
# def sim_cosine(p_idx, k=25):
#   if p_idx in Men['p_idx']: # p_idx가 Men에 들어가있으면
#     men_sim = cosine_men[p_idx].sort_values(ascending=False).index[:26]
#     recommend = product_c[product_c['p_idx'].isin(men_sim)]

#   else: # 다른 경우
#     women_sim = cosine_women[p_idx].sort_values(ascending=False).index[:26]
#     recommend = product_c[product_c['p_idx'].isin(women_sim)]

#   return recommend